In [9]:
import matplotlib.pyplot as plt

import fastf1
import fastf1.plotting
import pandas as pd

fastf1.plotting.setup_mpl(mpl_timedelta_support=False, misc_mpl_mods=False,
                          color_scheme='fastf1')


In [10]:
fastf1.events.get_event_schedule(year)

,RoundNumber,Country,Location,OfficialEventName,EventDate,EventName,EventFormat,Session1,Session1Date,Session1DateUtc,...,Session3,Session3Date,Session3DateUtc,Session4,Session4Date,Session4DateUtc,Session5,Session5Date,Session5DateUtc,F1ApiSupport
0,0,Bahrain,Sakhir,FORMULA 1 ARAMCO PRE-SEASON TESTING 2023,2023-02-25,Pre-Season Testing,testing,Practice 1,2023-02-23 10:00:00+03:00,2023-02-23 07:00:00,...,Practice 3,2023-02-25 10:00:00+03:00,2023-02-25 07:00:00,None,NaT,NaT,None,NaT,NaT,True
1,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2023,2023-03-05,Bahrain Grand Prix,conventional,Practice 1,2023-03-03 14:30:00+03:00,2023-03-03 11:30:00,...,Practice 3,2023-03-04 14:30:00+03:00,2023-03-04 11:30:00,Qualifying,2023-03-04 18:00:00+03:00,2023-03-04 15:00:00,Race,2023-03-05 18:00:00+03:00,2023-03-05 15:00:00,True
2,2,Saudi Arabia,Jeddah,FORMULA 1 STC SAUDI ARABIAN GRAND PRIX 2023,2023-03-19,Saudi Arabian Grand Prix,conventional,Practice 1,2023-03-17 16:30:00+03:00,2023-03-17 13:30:00,...,Practice 3,2023-03-18 16:30:00+03:00,2023-03-18 13:30:00,Qualifying,2023-03-18 20:00:00+03:00,2023-03-18 17:00:00,Race,2023-03-19 20:00:00+03:00,2023-03-19 17:00:00,True
3,3,Australia,Melbourne,FORMULA 1 ROLEX AUSTRALIAN GRAND PRIX 2023,2023-04-02,Australian Grand Prix,conventional,Practice 1,2023-03-31 12:30:00+10:00,2023-03-31 02:30:00,...,Practice 3,2023-04-01 12:30:00+10:00,2023-04-01 02:30:00,Qualifying,2023-04-01 16:00:00+10:00,2023-04-01 06:00:00,Race,2023-04-02 15:00:00+10:00,2023-04-02 05:00:00,True
4,4,Azerbaijan,Baku,FORMULA 1 AZERBAIJAN GRAND PRIX 2023,2023-04-30,Azerbaijan Grand Prix,sprint_shootout,Practice 1,2023-04-28 13:30:00+04:00,2023-04-28 09:30:00,...,Sprint Shootout,2023-04-29 12:30:00+04:00,2023-04-29 08:30:00,Sprint,2023-04-29 17:30:00+04:00,2023-04-29 13:30:00,Race,2023-04-30 15:00:00+04:00,2023-04-30 11:00:00,True
5,5,United States,Miami,FORMULA 1 CRYPTO.COM MIAMI GRAND PRIX 2023,2023-05-07,Miami Grand Prix,conventional,Practice 1,2023-05-05 14:00:00-04:00,2023-05-05 18:00:00,...,Practice 3,2023-05-06 12:30:00-04:00,2023-05-06 16:30:00,Qualifying,2023-05-06 16:00:00-04:00,2023-05-06 20:00:00,Race,2023-05-07 15:30:00-04:00,2023-05-07 19:30:00,True
6,6,Monaco,Monaco,FORMULA 1 GRAND PRIX DE MONACO 2023,2023-05-28,Monaco Grand Prix,conventional,Practice 1,2023-05-26 13:30:00+02:00,2023-05-26 11:30:00,...,Practice 3,2023-05-27 12:30:00+02:00,2023-05-27 10:30:00,Qualifying,2023-05-27 16:00:00+02:00,2023-05-27 14:00:00,Race,2023-05-28 15:00:00+02:00,2023-05-28 13:00:00,True
7,7,Spain,Barcelona,FORMULA 1 AWS GRAN PREMIO DE ESPAÑA 2023,2023-06-04,Spanish Grand Prix,conventional,Practice 1,2023-06-02 13:30:00+02:00,2023-06-02 11:30:00,...,Practice 3,2023-06-03 12:30:00+02:00,2023-06-03 10:30:00,Qualifying,2023-06-03 16:00:00+02:00,2023-06-03 14:00:00,Race,2023-06-04 15:00:00+02:00,2023-06-04 13:00:00,True
8,8,Canada,Montréal,FORMULA 1 PIRELLI GRAND PRIX DU CANADA 2023,2023-06-18,Canadian Grand Prix,conventional,Practice 1,2023-06-16 13:30:00-04:00,2023-06-16 17:30:00,...,Practice 3,2023-06-17 12:30:00-04:00,2023-06-17 16:30:00,Qualifying,2023-06-17 16:00:00-04:00,2023-06-17 20:00:00,Race,2023-06-18 14:00:00-04:00,2023-06-18 18:00:00,True
9,9,Austria,Spielberg,FORMULA 1 ROLEX GROSSER PREIS VON ÖSTERREICH 2023,2023-07-02,Austrian Grand Prix,sprint_shootout,Practice 1,2023-06-30 13:30:00+02:00,2023-06-30 11:30:00,...,Sprint Shootout,2023-07-01 12:00:00+02:00,2023-07-01 10:00:00,Sprint,2023-07-01 16:30:00+02:00,2023-07-01 14:30:00,Race,2023-07-02 15:00:00+02:00,2023-07-02 13:00:00,True


In [ ]:
YEARS = [2024]

acc = []

for year in YEARS:
  events = fastf1.events.get_event_schedule(year)

  for i, event in events.iterrows():
    if i != 1:
      continue

    if event.is_testing():
      continue

    session = fastf1.get_session(year, event['RoundNumber'], 'Race')
    session.load(telemetry=True)

    for driver in session.drivers:
      d = session.laps.pick_drivers(driver).get_pos_data().add_driver_ahead()

      mask = (df['DriverAhead'].shift() != df['DriverAhead']) & \
            (df['DriverAhead'].notna()) & \
            (df['DriverAhead'].shift().notna())

      # Include the first row if it has a driver ahead
      first_row_mask = df.index == 0
      if len(df) > 0 and pd.notna(df.loc[0, 'DriverAhead']):
          mask = mask | first_row_mask

      # Filter the DataFrame
      filtered_df = df[mask]

      dpp['DriverId'] = driver
      dpp['DriverName'] = session.get_driver(driver).FullName
      dpp['RoundNumber'] = event['RoundNumber']
      dpp['Country'] = event['Country']
      dpp['Location'] = event['Location']
      dpp['OfficialEventName'] = event['OfficialEventName']
      # dpp['EventDate'] = event['EventDate']
      dpp['EventName'] = event['EventName']
      dpp['EventFormat'] = event['EventFormat']
      
      acc.append(dpp)

df = pd.concat(acc, ignore_index=True)
df


core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


,Date,Status,X,Y,Z,Source,Time,SessionTime,DriverAhead,DistanceToDriverAhead,DriverId,DriverName,RoundNumber,Country,Location,OfficialEventName,EventName,EventFormat
0,2024-03-02 15:03:42.460,OnTrack,-280.0,3550.0,-157.0,pos,0 days 00:00:00.118000,0 days 01:00:00.029000,,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
1,2024-03-02 15:03:42.700,OnTrack,-280.0,3550.0,-157.0,pos,0 days 00:00:00.358000,0 days 01:00:00.269000,,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
2,2024-03-02 15:03:43.040,OnTrack,-280.0,3558.0,-157.0,pos,0 days 00:00:00.698000,0 days 01:00:00.609000,63,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
3,2024-03-02 15:03:43.460,OnTrack,-280.0,3574.0,-157.0,pos,0 days 00:00:01.118000,0 days 01:00:01.029000,4,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
4,2024-03-02 15:03:43.619,OnTrack,-279.0,3585.0,-157.0,pos,0 days 00:00:01.277000,0 days 01:00:01.188000,4,0.000000,1,Max Verstappen,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
427593,2024-03-02 16:35:46.778,OnTrack,-435.0,197.0,-159.0,pos,0 days 01:32:04.436000,0 days 02:32:04.347000,77,2288.916117,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
427594,2024-03-02 16:35:47.018,OnTrack,-421.0,470.0,-159.0,pos,0 days 01:32:04.676000,0 days 02:32:04.587000,77,2283.197228,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
427595,2024-03-02 16:35:47.378,OnTrack,-411.0,654.0,-159.0,pos,0 days 01:32:05.036000,0 days 02:32:04.947000,77,2274.697228,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional
427596,2024-03-02 16:35:47.638,OnTrack,-401.0,850.0,-159.0,pos,0 days 01:32:05.296000,0 days 02:32:05.207000,77,2268.600556,2,Logan Sargeant,1,Bahrain,Sakhir,FORMULA 1 GULF AIR BAHRAIN GRAND PRIX 2024,Bahrain Grand Prix,conventional


In [ ]:
session = fastf1.get_session(2023, 2, 'Race')
session.load(telemetry=True, weather=True)

core           INFO 	Loading data for Saudi Arabian Grand Prix - Race [v3.5.3]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core        WARNING 	Driver 11 completed the race distance 00:00.035000 before the recorded end of the session.
core           INFO 	Finished loading data for 20 drivers: ['11', '1', '14', '63', '44', '55', '16', '31', '10', '20', '

2023 Season Round 2: Saudi Arabian Grand Prix - Race

In [ ]:
print(df.to_json())